In [ ]:
import torch
import os
import matplotlib.pyplot as plt
from PIL import Image

!pip install git+https://github.com/openai/CLIP.git
import clip

In [ ]:
# -----------------------------------
# Setup Device and Load CLIP Model
# -----------------------------------
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load CLIP model and preprocessing pipeline
model, preprocess = clip.load("ViT-L/14", device=device)

# -----------------------------------
# Load Precomputed Mean and Whitening Matrix
# -----------------------------------
mean_path = 'mean_image_L14.pt'
w_mat_path = 'w_mat_image_L14.pt'

!wget --no-check-certificate "https://drive.google.com/uc?export=download&id=1fpdRdKjTg5FyhsbAPuP9D6gS5Xo-1b3n" -O w_mat_image_L14.pt
!wget --no-check-certificate "https://drive.google.com/uc?export=download&id=1XV04Ov7jsQnFc4MoBQop4flQFpRk-QyR" -O mean_image_L14.pt

# Load mean feature vector and whitening matrix
mean_features = torch.load(mean_path, map_location=device, weights_only=False)
w_mat = torch.load(w_mat_path, map_location=device, weights_only=False)

# Number of features (dimensionality)
N = len(mean_features)


In [ ]:
# -----------------------------------
# Load Images from Directory
# -----------------------------------
image_dir_path = 'images_hands'  # Directory containing images
os.makedirs(image_dir_path, exist_ok=True)

!wget --no-check-certificate "https://drive.google.com/uc?export=download&id=1WmhI93JzJijD9mEZLzLLfmBqAdwpsnqG" -O images_hands/137106.jpg
!wget --no-check-certificate "https://drive.google.com/uc?export=download&id=1s8uythPxDWq3ScmwI3umKN-Dc5NVFgtx" -O images_hands/001385.jpg
!wget --no-check-certificate "https://drive.google.com/uc?export=download&id=1NKnYI9gtJ3gHLAPZtff7hnLFei5b49Nn" -O images_hands/001178.png
!wget --no-check-certificate "https://drive.google.com/uc?export=download&id=1SP2gy3MX8MV2WWcdNT7i5hxgGiIYfjyV" -O images_hands/000798.png

image_names = os.listdir(image_dir_path)  # List all image filenames
images_path = [os.path.join(image_dir_path, image) for image in image_names]  # Construct full paths

In [ ]:
# -----------------------------------
# Process Images and Compute Log-Likelihood
# -----------------------------------
for image_path in images_path:
    # Load and preprocess image
    image = Image.open(image_path).convert("RGB")
    image_tensor = preprocess(image).unsqueeze(0).to(device)

    # Extract CLIP features
    image_features = model.encode_image(image_tensor).squeeze(0).to(device, dtype=torch.float32)

    # Center the features and apply whitening transformation
    cntr_features = image_features - mean_features
    w_features = torch.matmul(cntr_features, w_mat)

    # Compute log-likelihood
    log_like = -0.5 * (N * torch.log(torch.tensor(2 * torch.pi, device=device)) + torch.sum(w_features**2))

    # -----------------------------------
    # Plot Image with Log-Likelihood
    # -----------------------------------
    plt.figure(figsize=(6, 6))
    plt.imshow(image)
    plt.axis("off")  # Hide axis
    plt.title(f"Log-Likelihood: {log_like.item():.4f}", fontsize=14, color='red')